In [134]:
import pickle

import pandas as pd
import mlflow
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [135]:
from sklearn.pipeline import make_pipeline

In [136]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [137]:
df_train = read_dataframe('green_tripdata_2021-01.parquet')
df_val = read_dataframe('green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [138]:
with mlflow.start_run() as run:
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

MlflowException: API request to http://localhost:5000/api/2.0/mlflow/runs/create failed with exception HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/2.0/mlflow/runs/create (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f12b5662550>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
run.info.run_id

'f6bb6d8167984a7c84c3784d48231e1d'

RestException: RESOURCE_ALREADY_EXISTS: Registered Model (name=RandomForestRegression-staging-2) already exists. Error: (sqlite3.IntegrityError) UNIQUE constraint failed: registered_models.name
[SQL: INSERT INTO registered_models (name, creation_time, last_updated_time, description) VALUES (?, ?, ?, ?)]
[parameters: ('RandomForestRegression-staging-2', 1706454369506, 1706454369506, '')]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

MlflowException: The following failures occurred while downloading one or more artifacts from /home/jithish/Documents/zoomcamp/05/artifacts/1/141b763fa8294c9586c6452b5b34c94b/artifacts:
##### File preprocessor #####
[Errno 2] No such file or directory: '/home/jithish/Documents/zoomcamp/05/artifacts/1/141b763fa8294c9586c6452b5b34c94b/artifacts/preprocessor'